In [2]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [3]:
mlflow.set_experiment("Exp_3 - TfIdf Trigram max_features")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/434752975682846109', creation_time=1758539325608, experiment_id='434752975682846109', last_update_time=1758539325608, lifecycle_stage='active', name='Exp_3 - TfIdf Trigram max_features', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [4]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
X_train.shape

NameError: name 'X_train' is not defined

In [17]:
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigrams

    # Map labels to [0,1,2]
    y_raw = df['category'].map({-1: 2, 0: 0, 1: 1})

    # Train-test split
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        df['clean_comment'], y_raw, test_size=0.2, random_state=42, stratify=y_raw
    )

    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train_raw)
    X_test_vec = vectorizer.transform(X_test_raw)

    # -----------------------------
    # 3️⃣ Define KNN model
    # -----------------------------
    model = KNeighborsClassifier(
        n_neighbors=5,  # You can tune this parameter
        weights='uniform',
        algorithm='auto'
    )

    # -----------------------------
    # 4️⃣ Train and log everything with MLflow
    # -----------------------------
    with mlflow.start_run() as run:
        # Tags
        mlflow.set_tag("mlflow.runName", f"KNN_TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "KNeighborsClassifier")
        mlflow.set_tag("description", f"KNN with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer params
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log KNN params
        mlflow.log_param("n_neighbors", 5)
        mlflow.log_param("weights", "uniform")
        mlflow.log_param("algorithm", "auto")

        # Train
        model.fit(X_train_vec, y_train)

        # Predictions
        y_pred = model.predict(X_test_vec)

        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True, labels=[0,1,2])
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred, labels=[0,1,2])
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF max_features={max_features}")
        conf_matrix_file = f"confusion_matrix_{max_features}.png"
        plt.savefig(conf_matrix_file)
        mlflow.log_artifact(conf_matrix_file)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"knn_model_tfidf_max_features_{max_features}")

# -----------------------------
# 5️⃣ Run experiment for multiple max_features
# -----------------------------
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/09/25 23:26:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:27:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNN_TFIDF_Trigrams_max_features_1000 at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109/runs/bb1f50c91507475dbdbc61ca136de2bf
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109


2025/09/25 23:28:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:29:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNN_TFIDF_Trigrams_max_features_2000 at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109/runs/9bb7ff6742b24f818a7d80938885c829
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109


2025/09/25 23:30:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:31:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run KNN_TFIDF_Trigrams_max_features_8000 at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109/runs/dc7841c1394b4d68af278ae6be971867
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109


2025/09/25 23:43:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:43:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNN_TFIDF_Trigrams_max_features_9000 at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109/runs/462e6043723445669fff26cb450c24ff
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109


2025/09/25 23:45:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:45:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run KNN_TFIDF_Trigrams_max_features_10000 at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109/runs/f17e5d06271d4457abf5e50dd04fa2ff
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/434752975682846109
